In [1]:
from train_svm import load_ml_data

In [2]:
ws = [90, 90]
winL, winR = ws

do_preprocess=True
reduced_DS=False
maxRR=True
use_RR=True
norm_RR=True
compute_morph=['wvlt', 'HOS', 'lbp', 'OurMorph']

In [3]:
tr_features, tr_labels, eval_features, eval_labels = load_ml_data(
    (winL, winR),
    do_preprocess,
    reduced_DS,
    maxRR,
    use_RR,
    norm_RR,
    compute_morph,
    cross_patient=False,
    verbose=True)

Loading pickle: /home/congyu/dataset/ECG/mitdb/ml_learning/features/w_90_90_DS12_rm_bsline_maxRR_norm_RR_resample_10_lbp_hbf5_wvlt_HOS_MLII.pkl...
normalizing the data ... 


In [4]:
print(tr_features.shape)
print(tr_labels.shape)
print(eval_features.shape)
print(eval_labels.shape)

(80554, 78)
(80554,)
(20139, 78)
(20139,)


In [7]:
from utils import calc_class_weights
class_weights = calc_class_weights(tr_labels)

In [8]:
from sklearn import svm

In [9]:
svm_model = svm.SVC(C=0.01, 
                    kernel='rbf', 
                    degree=3, 
                    gamma='auto',
                    coef0=0.0, 
                    shrinking=True, 
                    probability=False, 
                    tol=0.001,
                    cache_size=200, 
                    class_weight=class_weights, 
                    max_iter=-1, 
                    decision_function_shape="ovo", 
                    random_state=None)

In [10]:
from utils import PrintTime

In [11]:
with PrintTime("training the model"):
    svm_model.fit(tr_features, tr_labels)


working on training the model ...
time for training the model: 568.87 sec



In [ ]:
from sklearn.metrics import f1_score, classification_report
from constant import AAMI_CLASSES

In [ ]:
y_pred = svm_model.predict(eval_features)

In [ ]:
print("marco f1 score", f1_score(eval_labels, y_pred, average='macro'))

In [ ]:
print(classification_report(eval_labels, y_pred, target_names=AAMI_CLASSES))

In [ ]:
model_svm_path="/home/congyu/dataset/ECG/mitdb/ml_learning/svm_models/baseline_model.joblib.pkl"

In [ ]:
import os
import joblib

In [ ]:
if model_svm_path:
    # Export model: save/write trained SVM model
    if not os.path.exists(os.path.dirname(model_svm_path)):
        os.makedirs(os.path.dirname(model_svm_path))
    joblib.dump(svm_model, model_svm_path)

In [ ]:
decision_ovo = svm_model.decision_function(eval_features)

In [ ]:
predict_ovo, counter = ovo_voting(decision_ovo, 4)

In [ ]:
perf_measures = compute_AAMI_performance_measures(predict_ovo, labels)
write_AAAMI_results_gamma(output_path, gamma_value, perf_measures, C_value, voting_strategy, DS)